In [1]:
import sys
from os.path import expanduser
home = expanduser("~")

sys.path.insert(0,home+"/fairing")
print(sys.path)

['/home/jovyan/fairing', '/opt/conda/lib/python36.zip', '/opt/conda/lib/python3.6', '/opt/conda/lib/python3.6/lib-dynload', '', '/opt/conda/lib/python3.6/site-packages', '/opt/conda/lib/python3.6/site-packages/object_detection-0.1-py3.6.egg', '/opt/conda/lib/python3.6/site-packages/IPython/extensions', '/home/jovyan/.ipython']


In [42]:
class TFserve(object):

    def __init__(self):
        print("init")
        self.homeDir = os.getcwd() + "/framework"
        sys.path.append( self.homeDir + "/TF_obj_detection_original/models/research/slim")
        #self.train_input = "ames_dataset/train.csv"
        #self.n_estimators = 50
        #self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None        
        
    def train(self):
        print("train")
        os.chdir(self.homeDir+"/TF_obj_detection_original/models/research/")

    def predict(self, X, feature_names):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return prediction.tolist()
        #rainer end
        

In [43]:
class HousingServe(object):
    
    def __init__(self):

        self.train_input = "ames_dataset/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None
        

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        #rainer start
        #return [[prediction.item(0), prediction.item(0)]]
        return prediction.tolist()
        #rainer end

In [44]:
tfs = TFserve()

init


In [45]:
tfs.train()

train


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/4stanislav/tfstuffwithkubeflow1/framework/TF_obj_detection_original/models/research/framework/TF_obj_detection_original/models/research/'

In [46]:
import os
import fairing

# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
#GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
#DOCKER_REGISTRY = 'mtr.external.otc.telekomcloud.com/rainer_englisch/kubeflow_test1'
DOCKER_REGISTRY = 'index.docker.io/m1st3rb3an'
PY_VERSION = ".".join([str(x) for x in sys.version_info[0:3]])
BASE_IMAGE = 'library/python:{}'.format(PY_VERSION)
BASE_IMAGE = 'm1st3rb3an/rainers-python:latest'

In [48]:
from fairing import TrainJob
#from fairing.backends import KubeflowGKEBackend
from fairing.backends import KubeflowBackend
train_job = TrainJob(TFserve, BASE_IMAGE, input_files=['ames_dataset/train.csv', "requirements.txt"],
                     docker_registry=DOCKER_REGISTRY, backend=KubeflowBackend())
train_job.submit()

W0807 08:32:28.993108 139800470828864 tasks.py:31] Using preprocessor: <class 'fairing.preprocessors.function.FunctionPreProcessor'>
W0807 08:32:28.993858 139800470828864 tasks.py:34] Using docker registry: index.docker.io/m1st3rb3an
W0807 08:32:28.995535 139800470828864 tasks.py:46] Using builder: <class 'fairing.builders.append.append.AppendBuilder'>
I0807 08:32:28.996164 139800470828864 tasks.py:49] Building the docker image.
W0807 08:32:28.996930 139800470828864 append.py:57] Building image using Append builder...
I0807 08:32:28.998320 139800470828864 base.py:89] Creating docker context: /tmp/fairing_context_l_g8klb_
W0807 08:32:28.999423 139800470828864 base.py:82] /home/jovyan/fairing/fairing/__init__.py already exists in Fairing context, skipping...


FileNotFoundError: [Errno 2] No such file or directory: 'ames_dataset/train.csv'